Purpose of the Notebook
The logger.ipynb notebook is designed to handle logging and time management for experiments in a deep learning project. It likely includes functionality to:


---
1. Function: get_root_logger
Purpose:
To initialize and retrieve a root logger for logging messages in an application. It ensures consistent logging setup with options for console and file output.
Functionality:
Checks if a logger with the specified name exists; if so, returns it.
Adds a StreamHandler for console output by default.
Adds a FileHandler if a log_file is specified, directing logs to a file.
Formats log messages with a timestamp, log level, and message content.
Allows setting the logging level to control message severity.
Example Usage:
)
---

2. Class: MessageLogger
Purpose:
Logs formatted messages for deep learning experiments, including progress and performance metrics.
Functionality:
Initializes with experiment configuration, including name, logging interval, and maximum iterations.
Formats log messages with epoch, iteration, learning rates, and performance metrics.
Calculates and logs elapsed time and estimated time remaining.
Optionally logs metrics to TensorBoard if enabled.
Example Usage:
)
---
3. Function: init_tb_logger
Purpose:
Initializes a TensorBoard logger for visualizing metrics during training.
Functionality:
Uses PyTorch's SummaryWriter to create a logger that writes logs to a specified directory.
Returns the initialized SummaryWriter for use in logging metrics.
Example Usage:
)


In [1]:
import datetime  # Provides classes for manipulating dates and times.
import logging   # Provides a flexible framework for emitting log messages from Python programs.
import time      # Provides various time-related functions, such as getting the current time.

## Class: MessageLogger
### Purpose: Logs formatted messages for deep learning experiments, including progress and performance metrics. MessageLogger class during a training loop. This example shows how to log information such as epoch, iteration, learning rates, and performance metrics like loss.


In [ ]:
# Class: MessageLogger
# Purpose: Logs formatted messages for deep learning experiments, including progress and performance metrics.

class MessageLogger():
    """Message logger for printing.

    Args:
        opt (dict): Config. It contains the following keys:
            name (str): Experiment name.
            logger (dict): Contains 'print_freq' (str) for logger interval.
            train (dict): Contains 'niter' (int) for total iterations.
            use_tb_logger (bool): Use tensorboard logger.
        start_iter (int): Start iteration. Default: 1.
        tb_logger (obj:`tb_logger`): Tensorboard logger. Default: None.
    """

    def __init__(self, opt, start_iter=1, tb_logger=None):
        # Initialize logger with experiment configuration
        self.exp_name = opt['name']
        self.interval = opt['print_freq']
        self.start_iter = start_iter
        self.max_iters = opt['max_iters']
        self.use_tb_logger = opt['use_tb_logger']
        self.tb_logger = tb_logger
        self.start_time = time.time()  # Record the start time
        self.logger = get_root_logger()  # Get the root logger

    def __call__(self, log_vars):
        """Format logging message.

        Args:
            log_vars (dict): Contains the following keys:
                epoch (int): Epoch number.
                iter (int): Current iteration.
                lrs (list): List of learning rates.
                time (float): Iteration time.
                data_time (float): Data loading time for each iteration.
        """
        # Extract epoch, iteration, and learning rates
        epoch = log_vars.pop('epoch')
        current_iter = log_vars.pop('iter')
        lrs = log_vars.pop('lrs')

        # Construct the log message
        message = (f'[{self.exp_name[:5]}..][epoch:{epoch:3d}, '
                   f'iter:{current_iter:8,d}, lr:(')
        for v in lrs:
            message += f'{v:.3e},'
        message += ')] '

        # Add time and estimated time to the message
        if 'time' in log_vars.keys():
            iter_time = log_vars.pop('time')
            data_time = log_vars.pop('data_time')

            total_time = time.time() - self.start_time
            time_sec_avg = total_time / (current_iter - self.start_iter + 1)
            eta_sec = time_sec_avg * (self.max_iters - current_iter - 1)
            eta_str = str(datetime.timedelta(seconds=int(eta_sec)))
            message += f'[eta: {eta_str}, '
            message += f'time: {iter_time:.3f}, data_time: {data_time:.3f}] '

        # Add other log variables, especially losses
        for k, v in log_vars.items():
            message += f'{k}: {v:.4e} '
            # Log to TensorBoard if enabled
            if self.use_tb_logger and 'debug' not in self.exp_name:
                self.tb_logger.add_scalar(k, v, current_iter)

        # Output the log message
        self.logger.info(message)

# Example Usage
# Assuming `opt` is a dictionary with the necessary configuration
opt = {
    'name': 'experiment_1',
    'print_freq': 10,
    'max_iters': 1000,
    'use_tb_logger': True
}

# Initialize the logger
message_logger = MessageLogger(opt)

# Simulate logging during training
log_vars = {
    'epoch': 1,
    'iter': 10,
    'lrs': [0.001],
    'time': 0.5,
    'data_time': 0.1,
    'loss': 0.02
}

# Log the message
message_logger(log_vars)

# Function: init_tb_logger
# Purpose: Initialize a TensorBoard logger for visualizing metrics during training.


In [ ]:
def init_tb_logger(log_dir):
    """
    Initialize a TensorBoard logger.

    Args:
        log_dir (str): The directory where TensorBoard logs will be saved.

    Returns:
        SummaryWriter: An instance of TensorBoard's SummaryWriter for logging.
    """
    from torch.utils.tensorboard import SummaryWriter  # Import the SummaryWriter class from PyTorch's TensorBoard utilities
    tb_logger = SummaryWriter(log_dir=log_dir)  # Create a SummaryWriter instance with the specified log directory
    return tb_logger  # Return the initialized TensorBoard logger

In [ ]:
# def init_tb_logger(log_dir):
#     from torch.utils.tensorboard import SummaryWriter
#     tb_logger = SummaryWriter(log_dir=log_dir)
#     return tb_logger


# Function: get_root_logger
# Purpose: Initialize and retrieve the root logger for logging messages in the application.

The `get_root_logger` function is designed to initialize and retrieve a root logger for logging messages in an application. It ensures that logging is set up with a consistent format and level, and it can direct log output to both the console and a file if specified.

In [ ]:
def get_root_logger(logger_name='base', log_level=logging.INFO, log_file=None):
    """Get the root logger.

    The logger will be initialized if it has not been initialized. By default, a
    StreamHandler will be added. If `log_file` is specified, a FileHandler will
    also be added.

    Args:
        logger_name (str): The name of the root logger. Default is 'base'.
        log_file (str | None): The log filename. If specified, a FileHandler
            will be added to the root logger to write logs to a file.
        log_level (int): The logging level for the root logger. Default is INFO.

    Returns:
        logging.Logger: The configured root logger.
    """
    # Retrieve the logger with the specified name
    logger = logging.getLogger(logger_name)
    
    # If the logger already has handlers, return it to avoid reinitialization
    if logger.hasHandlers():
        return logger

    # Define the log message format
    format_str = '%(asctime)s.%(msecs)03d - %(levelname)s: %(message)s'
    # Configure the basic logging settings with the specified format and level
    logging.basicConfig(format=format_str, level=log_level)

    # If a log file is specified, add a FileHandler to write logs to the file
    if log_file is not None:
        file_handler = logging.FileHandler(log_file, 'w')
        file_handler.setFormatter(logging.Formatter(format_str))
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    # Return the configured logger
    return logger

import logging

# Initialize the root logger with a specific name, level, and optional log file
logger = get_root_logger(logger_name='my_app', log_level=logging.DEBUG, log_file='app.log')

# Log messages at various levels
logger.debug("This is a debug message.")
logger.info("This is an info message.")
logger.warning("This is a warning message.")
logger.error("This is an error message.")
logger.critical("This is a critical message.")

In [7]:
# def get_root_logger(logger_name='base', log_level=logging.INFO, log_file=None):
#     """Get the root logger.

#     The logger will be initialized if it has not been initialized. By default a
#     StreamHandler will be added. If `log_file` is specified, a FileHandler will
#     also be added.

#     Args:
#         logger_name (str): root logger name. Default: base.
#         log_file (str | None): The log filename. If specified, a FileHandler
#             will be added to the root logger.
#         log_level (int): The root logger level. Note that only the process of
#             rank 0 is affected, while other processes will set the level to
#             "Error" and be silent most of the time.

#     Returns:
#         logging.Logger: The root logger.
#     """
#     logger = logging.getLogger(logger_name)
#     # if the logger has been initialized, just return it
#     if logger.hasHandlers():
#         return logger

#     format_str = '%(asctime)s.%(msecs)03d - %(levelname)s: %(message)s'
#     logging.basicConfig(format=format_str, level=log_level)

#     if log_file is not None:
#         file_handler = logging.FileHandler(log_file, 'w')
#         file_handler.setFormatter(logging.Formatter(format_str))
#         file_handler.setLevel(log_level)
#         logger.addHandler(file_handler)

#     return logger
